In [ ]:
import numpy as np
import pandas as pd
import json
import gc
import os
import time
from datetime import datetime

In [ ]:
l = os.listdir()
l = [i for i in l if '.csv' in i]
l

In [ ]:
review = pd.read_csv('review.csv',index_col=0)
review = review[['business_id','user_id','review_id','text','cool','funny','useful']]

In [ ]:
# Drop na values
review.dropna(inplace=True)

In [ ]:
space = []
for i, t in enumerate(review['text']):
    if t.isspace():
        space.append(i)
space
# No space inside

### Unsupervised Sentiment Analysis with Clustering Algorithm

The main idea behind this approach is that negative and positive words usually are surrounded by similar words.

https://towardsdatascience.com/unsupervised-sentiment-analysis-a38bf1906483

https://github.com/rafaljanwojcik/Unsupervised-Sentiment-Analysis/blob/master/preprocessing_and_embeddings/Preprocessing_and_Embeddings.ipynb

In [ ]:
from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec
from gensim.test.utils import get_tmpfile
from gensim.models import KeyedVectors
import multiprocessing

In [ ]:
sent = [r for r in review.text]

In [ ]:
# Create Bigram for later training
# https://www.kaggle.com/hamishdickson/training-and-plotting-word2vec-with-bigrams
# https://stackoverflow.com/questions/46129335/get-bigrams-and-trigrams-in-word2vec-gensim
print(datetime.now())
phrases = Phrases(sent, min_count=2)
print(datetime.now())
bigram = Phraser(phrases)
print(datetime.now())
sentences = bigram[sent]
print(datetime.now())

In [ ]:
sent = [row for row in review.title]
phrases = Phrases(sent, min_count=1, progress_per=50000)
bigram = Phraser(phrases)
sentences = bigram[sent]
sentences[1]

In [ ]:
w2v_model = Word2Vec(min_count=3,
                     window=5,
                     size=300,
                     sample=1e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=multiprocessing.cpu_count()-1)

min count = 3 - remove most unusual words from training embeddings, like words 'ssssuuuuuuuppppppeeeeeerrrr', which actually stands for 'super', and doesn't need additional training

window = 4 - Word2Vec model will learn to predict given word from up to 4 words to the left, and up to 4 words to the right

size = 300 - size of hidden layer used to predict surroundings of embedded word, which also stands for dimensions of trained embeddings

sample = 1e-5 - probability baseline for subsampling most frequent words from surrounding of embedded word

negative = 20 - number of negative (ones that shouldn't have been predicted while modeling selected pair of words) words that will have their corresponding weights updated while training on specific training example, along with positive word

In [ ]:
w2v_model.build_vocab(sent, progress_per=50000)

In [ ]:
w2v_model.train(sent, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

In [ ]:
w2v_model.init_sims(replace=True)
w2v_model.save("word2vec.model")